In [52]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import datetime as dt
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
#from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [53]:
# Creating executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\sethi\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [54]:
#1 Reading Mars information from NASA
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [55]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


In [56]:
#2 Reading title
results = soup.find('ul', class_='item_list')
results2=results.find('li', class_='slide')
news_title = results2.find("div", class_="content_title").text

news_title

"5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover"

In [57]:
#Scraping new teaser
news_p=results2.find('div', class_='article_teaser_body').text
news_p

"The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative."

In [58]:
#Getting the image url
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [59]:
browser.links.find_by_partial_text('FULL IMAGE').click()
browser.links.find_by_partial_text('more info').click()
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [60]:
results = soup.find('figure', class_='lede')
results2=results.a['href']


In [61]:
results2

'/spaceimages/images/largesize/PIA17474_hires.jpg'

In [62]:
featured_image_url= 'https://www.jpl.nasa.gov' + results2

In [63]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17474_hires.jpg'

In [64]:
#Creating a dictionary to insert in Mongo
heading_data = zip(news_title, news_p, featured_image_url)
hd_dt = {}
    
hd_dt['news_title'] = news_title
    

hd_dt['news_p'] = news_p
    
hd_dt['featured_image_url'] = featured_image_url

hd_dt

{'news_title': "5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover",
 'news_p': "The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative.",
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17474_hires.jpg'}

In [65]:
# # Creating executable path
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

In [66]:
#Getting hemisphere picture and information
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [67]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [68]:
results3=soup.find_all('div', class_='description')
results3


[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>,
 <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p></div>,
 <div class="description"><a 

In [69]:
#Getting heading new and image url
heading_list=[]
url_list=[]

for result in results3:
    heading=result.find('h3').text
    heading_list.append(heading)
    url=result.find('a')['href']
    url_list.append(url)
    new_url=['https://astrogeology.usgs.gov'+ x for x in url_list ]


new_url


['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [70]:
heading_list

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [71]:

new_url

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [72]:
# # Creating executable path
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

full_imgs=[]

for new in new_url:
    url=new
    browser.visit(url)
    browser.links.find_by_partial_text('Sample').click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results6 = soup.find_all('img', class_='wide-image')
    img_path = results6[0]['src']
    new_img = 'https://astrogeology.usgs.gov/' + img_path
    
# full image links to a list
    full_imgs.append(new_img)


In [73]:
results6


[<img class="wide-image" src="/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg"/>]

In [74]:
img_path

'/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'

In [75]:
full_imgs

['https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [76]:
heading_list

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [77]:
heading_url = zip(heading_list, full_imgs)

hemisphereurl = []

# Iterate through the zipped object
for x,y in heading_url:
    
    hemi_dict = {}
    
    # Add hemisphere title to dictionary
    hemi_dict['title'] = x
    
    # Add image url to dictionary
    hemi_dict['img_url'] = y
    
    # Append the list with dictionaries
    hemisphereurl.append(hemi_dict)

hemisphereurl

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [78]:
browser.quit()

In [79]:
#Reading Table form space facts 

import pandas as pd

In [80]:
url = 'https://space-facts.com/mars/'

In [81]:
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [82]:
df = tables[0]

df.columns = ['Mars Profile', 'Stats']
# df.set_index('Mars Profile', inplace=True)
df

,Mars Profile,Stats
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [83]:
type(df)
tab_html=df.to_html
print(tab_html)

<bound method DataFrame.to_html of            Mars Profile                          Stats
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers>


In [84]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [85]:
#Connecting to DB and showing the listing
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [86]:
db = client.mars_app
collection = db.listing
collection1 = db.heading

In [87]:
collection1.insert_one(hd_dt)

In [95]:
collection.insert_many(hemisphereurl)

In [96]:
listings = db.listing.find()

for listing in listings:
    print(listing)

{'_id': ObjectId('5fd4592be7ff38111c7be41e'), 'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}
{'_id': ObjectId('5fd4592be7ff38111c7be41f'), 'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}
{'_id': ObjectId('5fd4592be7ff38111c7be420'), 'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}
{'_id': ObjectId('5fd4592be7ff38111c7be421'), 'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}


In [94]:
#db.listing.delete_many({})

In [91]:
#db.heading.delete_many({})

In [92]:
# text_file = open(".\templates\index.html", "w")
# text_file.write(tab_html)
# text_file.close()

In [93]:
#print(tables)